In [1]:
# === catboost_train_score_numeric.py ===
import numpy as np
import pandas as pd
from typing import Dict, Any, Optional, List
from catboost import CatBoostClassifier, Pool

IGNORED_COLS_DEFAULT = ["filepath", "path"]

def _auto_class_weights(y: pd.Series) -> Optional[List[float]]:
    """
    Si hay desbalance, calcula pesos inversamente proporcionales a la frecuencia.
    Si está balanceado, regresa None.
    """
    vc = y.value_counts(normalize=True)
    if vc.min() < 0.4:  # umbral simple de desbalance
        # Importante: mantener el orden de clases como strings/nums ordenados por valor
        # CatBoost mapeará internamente; usaremos model.classes_ luego.
        return (1.0 / vc).tolist()
    return None

def _select_numeric_features(df: pd.DataFrame, drop_cols: List[str], target_col: Optional[str]) -> List[str]:
    cols = df.columns.tolist()
    cols = [c for c in cols if c not in (drop_cols or [])]
    if target_col and target_col in cols:
        cols.remove(target_col)
    num_cols = df[cols].select_dtypes(include=[np.number]).columns.tolist()
    if not num_cols:
        raise ValueError("No se encontraron columnas numéricas después de excluir filepath/path/target.")
    return num_cols

def train_and_score_catboost_numeric(
    train_df: pd.DataFrame,
    score_df: pd.DataFrame,
    target_col: str = "clase",
    drop_cols: Optional[List[str]] = None,
    params: Optional[Dict[str, Any]] = None,
    model_path: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Entrena CatBoostClassifier con SOLO features numéricas y predice sobre score_df.
    - Usa TODO el train (sin early stopping ni holdout).
    - Ignora columnas 'filepath' y 'path' (configurable en drop_cols).
    - Alinea y ordena columnas numéricas iguales entre train y score.
    - Devuelve: modelo, columnas usadas, y un DF con predicciones sobre score (y probs).
    """
    drop_cols = drop_cols or IGNORED_COLS_DEFAULT

    # Validaciones básicas
    if target_col not in train_df.columns:
        raise ValueError(f"'{target_col}' no está en train_df.")
    if target_col in score_df.columns:
        # No es grave, pero evitamos fugas por si alguien la dejó
        score_df = score_df.drop(columns=[target_col]).copy()

    # 1) Selección de columnas numéricas (tras ignorar filepath/path y target)
    train_num_cols = _select_numeric_features(train_df, drop_cols, target_col)
    score_num_cols = _select_numeric_features(score_df, drop_cols, target_col=None)

    # 2) Intersección y orden consistente de columnas
    shared_num_cols = sorted(list(set(train_num_cols).intersection(score_num_cols)))
    if not shared_num_cols:
        raise ValueError("Train y Score no comparten columnas numéricas después de los filtros.")
    # Mantén un orden estable (alfabético o como en train). Elegimos como en train:
    shared_num_cols = [c for c in train_num_cols if c in shared_num_cols]

    X_train = train_df[shared_num_cols]
    y_train = train_df[target_col]
    X_score = score_df[shared_num_cols]

    # 3) Parámetros (sin early stopping; entrenar con TODO el train)
    default_params = dict(
        loss_function="MultiClass" if y_train.nunique() > 2 else "Logloss",
        eval_metric="MultiClass" if y_train.nunique() > 2 else "AUC",
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=10.0,
        iterations=2000,          # puedes subir/bajar
        random_seed=42,
        verbose=False,
        use_best_model=False,     # <- importante: sin holdout
        class_weights=None        # se setea abajo si aplica
    )

    # Ponderación de clases automática si hay desbalance
    cw = _auto_class_weights(y_train)
    if cw is not None:
        default_params["class_weights"] = cw

    if params:
        default_params.update(params)

    # 4) Entrenar
    train_pool = Pool(X_train, y_train)  # CatBoost maneja NaNs numéricos nativamente
    model = CatBoostClassifier(**default_params)
    model.fit(train_pool)

    # 5) Predecir en score
    score_pool = Pool(X_score)
    y_pred = model.predict(score_pool)
    # y_pred es una matriz Nx1; aplanamos
    y_pred = np.array(y_pred).ravel()

    # Probabilidades por clase (si el modelo las soporta)
    prob_cols = []
    try:
        proba = model.predict_proba(score_pool)  # shape: (n, n_classes) o (n, 2) binario
        classes_ = model.classes_ if hasattr(model, "classes_") else np.unique(y_train)
        # Crear columnas p_<clase>
        prob_cols = [f"p_{str(c)}" for c in classes_]
    except Exception:
        proba = None

    # 6) Armar salida de predicción
    out_cols = []
    for c in drop_cols:
        if c in score_df.columns:
            out_cols.append(c)

    preds_df = pd.DataFrame(index=score_df.index)
    for c in out_cols:
        preds_df[c] = score_df[c]

    preds_df["y_pred"] = y_pred

    if proba is not None:
        # Asegurar que las columnas sigan el orden de classes_
        for j, cname in enumerate(prob_cols):
            preds_df[cname] = proba[:, j]

    # 7) Guardar modelo si se solicita
    if model_path:
        model.save_model(model_path)

    return {
        "model": model,
        "used_numeric_features": shared_num_cols,
        "preds_score": preds_df
    }



In [6]:
# Ejemplo de cómo llamarlo con tus dfs ya cargados:
train_df = pd.read_parquet(r'data/processed/features_wavs_train.parquet')  # tiene 'clase'
score_df = pd.read_parquet(r'data/processed/features_wavs_test.parquet')  # NO tiene 'clase'

out = train_and_score_catboost_numeric(
    train_df=train_df,
     score_df=score_df,
     target_col="clase",
     drop_cols=["filepath", "path"],
     params=dict(iterations=1500, depth=5, l2_leaf_reg=12.0),
     model_path="modelo_catboost.cbm"
 )
print("Features usadas:", len(out["used_numeric_features"]))
print(out["preds_score"])



Features usadas: 195
                           filepath  y_pred       p_1       p_2       p_3  \
0    data/raw/nuria_wavs_test\1.wav       4  0.001230  0.002944  0.002085   
1   data/raw/nuria_wavs_test\10.wav       2  0.023944  0.784392  0.030600   
2   data/raw/nuria_wavs_test\11.wav       4  0.004360  0.010166  0.007099   
3   data/raw/nuria_wavs_test\12.wav       2  0.007679  0.933188  0.009296   
4   data/raw/nuria_wavs_test\13.wav       2  0.010171  0.899660  0.013021   
5   data/raw/nuria_wavs_test\14.wav       5  0.000711  0.001690  0.001273   
6   data/raw/nuria_wavs_test\15.wav       5  0.003805  0.007122  0.006297   
7   data/raw/nuria_wavs_test\16.wav       2  0.026215  0.806194  0.030805   
8   data/raw/nuria_wavs_test\17.wav       5  0.053532  0.060666  0.328191   
9   data/raw/nuria_wavs_test\18.wav       4  0.005962  0.012428  0.009665   
10  data/raw/nuria_wavs_test\19.wav       1  0.341174  0.077131  0.181811   
11   data/raw/nuria_wavs_test\2.wav       5  0.010653  

In [8]:
preds = out["preds_score"]
solo_cols = preds[["filepath", "y_pred"]]


In [9]:
solo_cols

,filepath,y_pred
0,data/raw/nuria_wavs_test\1.wav,4
1,data/raw/nuria_wavs_test\10.wav,2
2,data/raw/nuria_wavs_test\11.wav,4
3,data/raw/nuria_wavs_test\12.wav,2
4,data/raw/nuria_wavs_test\13.wav,2
5,data/raw/nuria_wavs_test\14.wav,5
6,data/raw/nuria_wavs_test\15.wav,5
7,data/raw/nuria_wavs_test\16.wav,2
8,data/raw/nuria_wavs_test\17.wav,5
9,data/raw/nuria_wavs_test\18.wav,4
